In [1]:
import pandas as pd
from gridoptim import run_optimizer
pd.set_option("display.max_columns", None)

In [2]:
run_optimizer(
    checkpoint=1,
    counter=8001,
    start=1,
    end=5000,
    tickers=['EUR_USD'],
    frequency=['M5'],
    init_bal=[1000],
    init_trade_size=[1000],
    grid_pips=[10],
    sl_grid_count=[50],
    stop_loss=['margin_grid_count'],
    margin_sl_percent=[0.90],
    sizing=['dynamic'],
    cash_out_factor=[1.1],
    INPUTS_FILE='inputs.8a.csv'
)

sim_name           init_trade_size    grid_pips  stop_loss            sl_grid_count    stoploss_pips    margin_sl_percent  sizing      cash_out_factor
EUR_USD-M5-8001               1000           10  margin_grid_count               50              500                  0.9  dynamic                 1.1


 Simulating... : 100%|██████████| 4999/4999 [00:05<00:00, 903.58it/s] 


In [3]:
# run_optimizer(
#     checkpoint=1,
#     counter=8001,
#     start=None,
#     end=None,
#     tickers=['EUR_USD'],
#     frequency=['M5'],
#     init_bal=[1000],
#     init_trade_size=[1000],
#     grid_pips=[10, 20, 30, 40, 50, 60, 100],
#     sl_grid_count=[5, 10, 15, 20, 30, 50, 100],
#     stop_loss=['margin_grid_count'],
#     margin_sl_percent=[0.90, 0.80, 0.70],
#     sizing=['dynamic', 'static'],
#     cash_out_factor=[1.1, 1.25, 1.5, 1.75],
#     INPUTS_FILE='inputs.8.csv'
# )

In [106]:
MC_PERCENT = 0.50
margin_rate = 0.03333333333333
def calc_trade_size(i: int, 
               init_bal: float, 
               init_trade_size: int, 
               net_bal: float, 
               margin_used: float, 
               sizing: str, 
               sl_pip_position: float, 
               tp_pips: int):
    sizing_ratio = init_trade_size / init_bal
    if i == 0:
        trade_size = init_trade_size
    else:
        calc_trade_size = int(net_bal * sizing_ratio) if sizing == 'dynamic' else init_trade_size
        sl_cover_trade_size = sl_pip_position / tp_pips / 2
        required_trade_size = max(calc_trade_size, sl_cover_trade_size)
        allowed_trade_size = max(0, (net_bal / MC_PERCENT- margin_used) / (2 * margin_rate))
        trade_size = round(min(required_trade_size, allowed_trade_size), 2)
        print('calc_trade_size', calc_trade_size)
        print('sl_cover_trade_size', sl_cover_trade_size)
        print('required_trade_size', required_trade_size)
        print('allowed_trade_size', allowed_trade_size)
        print('trade_size', trade_size)
    return trade_size

In [117]:
margin_rate = 0.03333333333333
def calc_trade_size(i: int, 
               init_bal: float, 
               init_trade_size: int, 
               net_bal: float, 
               margin_used: float, 
               sizing: str, 
               sl_pip_position: float, 
               tp_pips: int,
               uncovered_pip_position: int,
               margin_sl_percent: float):
    sizing_ratio = init_trade_size / init_bal
    if i == 0:
        trade_size = init_trade_size
    else:
        calc_trade_size = int(net_bal * sizing_ratio) if sizing == 'dynamic' else init_trade_size
        if uncovered_pip_position > 0:
            sl_cover_trade_size = uncovered_pip_position / tp_pips / 2
            required_trade_size = max(calc_trade_size, sl_cover_trade_size)
            allowed_trade_size = max(0, (net_bal / margin_sl_percent - margin_used) / (2 * margin_rate))
            trade_size = int(min(required_trade_size, allowed_trade_size))
        else:
            trade_size = calc_trade_size
        print('calc_trade_size', calc_trade_size)
        print('sl_cover_trade_size', sl_cover_trade_size)
        print('required_trade_size', required_trade_size)
        print('allowed_trade_size', allowed_trade_size)
        print('trade_size', trade_size)
    return trade_size

In [119]:
i=1
init_bal=1000
init_trade_size=100
net_bal=1063.15
margin_used=31.4
sizing='dynamic'
tp_pips=30
grids=10
prev_trade_size=3000
sl_pip_position=tp_pips*grids*prev_trade_size
uncovered_pip_position=1752178.4
margin_sl_percent=0.8
trade_size = calc_trade_size(i=1,
                            init_bal=init_bal, 
                            init_trade_size=init_trade_size, 
                            net_bal=net_bal, 
                            margin_used=margin_used, 
                            sizing=sizing, 
                            sl_pip_position=sl_pip_position, 
                            tp_pips=tp_pips,
                            uncovered_pip_position=uncovered_pip_position,
                            margin_sl_percent=margin_sl_percent)

calc_trade_size 106
sl_cover_trade_size 29202.97333333333
required_trade_size 29202.97333333333
allowed_trade_size 19463.062500001943
trade_size 19463


In [108]:
req_margin = round((trade_size + 0) * margin_rate * 2, 2)
net_bal >= (req_margin + margin_used) * MC_PERCENT

False

In [109]:
req_margin = round((trade_size + 1) * margin_rate * 2, 2)
net_bal >= (req_margin + margin_used) * MC_PERCENT

False

In [ ]:
print(True)    